In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
import time
import copy
import pandas as pd
import seaborn as sns
from IPython.display import clear_output
from sklearn.utils import class_weight

from config_reader import Config
from read_data import read_data
from preproc_data import *
from label_data import label_data
from split_data import split_data
from detection import *

cfg = Config()
%load_ext autoreload
%autoreload 2

In [2]:
cyclone_events, data = read_data(path='../data')
# Consider renaming first variable
cyclone_events_data, data, metrics = preproc_data(cyclone_events, metrics=data)
labels, events = label_data(cyclone_events_data.ce2)
train_data, test_data, labels_train, labels_test, train_id, test_id = split_data(data, labels, ratio=0.8)

In [3]:
labels_train_old = labels_train
labels_test_old = labels_test
labels_train = [[item, 1 - item] for item in labels_train]
labels_test = [[item, 1 - item] for item in labels_test]

In [4]:
desc = "ResNet18_2"
filename = "ResNet18_2.ipynb"
network_name = filename.split('.')[0]
comment = ""

In [6]:
from show_test_data import show_test_data
import torch
import torch.nn as nn
import numpy as np
from config_reader import Config
from sigma import sigma
import pandas as pd

def get_start_ticks(labels_test):
    labels_test = np.array(labels_test)
    start_ticks = []
    for i in range(len(labels_test) - 1):
        if (labels_test[i] == 0 and labels_test[i+1] == 1):
            start_ticks.append(i+1)
    return np.array(start_ticks)

def predicted_event_id(tick, start_ticks):
    idx = (np.abs(start_ticks - tick)).argmin()
    return idx

def fill_preicted_events(predicted_events, start_ticks, labels_test):
    cfg = Config()
    #res = np.zeros(shape=len(labels_test), dtype=int)
    res = [0] * len(labels_test)
    for k, pr in enumerate(predicted_events):
        if pr == 1:
            for i in range(start_ticks[k], start_ticks[k] + cfg.w):
                res[i] = 1

    res = indices(res, 1)
    return np.array(res, dtype=int)

def indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]


def test(nn, test_data, labels_test):
    # Test the model
    cfg = Config()
    nn.eval()    
    sigmas = []
    tp_ids = []
    fp_ids = []
    fn_ids = []
    tn_ids = []

    if cfg.mode == 'predict':
        predicted = np.zeros(shape=test_data.shape[2])

    with torch.no_grad():
        tp = 0
        fp = 0
        fn = 0
        tn = 0
        for i in range(test_data.shape[2]):
            if (labels_test[i] == 2):
                continue
            image = test_data[:, :, i]
            label = labels_test[i]
            
            image = image.unsqueeze(0) 
            image = image.unsqueeze(0) 
            
            test_output = nn(image)
            #arg = test_output.item()
            #sig = sigma(arg)
            #sigmas.append(sig)
            
            _, pred_y = torch.max(test_output.data, 1)
            # print(f"pred_y {pred_y}")
            # print(f"label {label}")
            #print(sig)
            # 0 - 1 when cyclone is present
            if (pred_y == label and label == 1):
                tp += 1
                tp_ids.append(i)
            # 1 - 0 when no cyclone is present
            elif (pred_y == label and label == 0):
                tn += 1
                tn_ids.append(i)
            # 0 - 1 when no cyclone is present
            elif (pred_y != label and label == 1):
                fn += 1
                fn_ids.append(i)
            # 1 - 0 when cyclone is present
            elif (pred_y != label and label == 0):
                fp += 1
                fp_ids.append(i)
            pass   
        pass
    
    if cfg.mode == 'predict':
        predicted[tp_ids] = 1
        event_start_ticks = get_start_ticks(labels_test)
        predicted_events = np.zeros(shape=len(event_start_ticks))

        for item in tp_ids:
            predicted_event = predicted_event_id(item, event_start_ticks)
            predicted_events[predicted_event] = 1

        tp_ids = fill_preicted_events(predicted_events, event_start_ticks, labels_test)

        tp = predicted_events.sum()
        fn = len(event_start_ticks) - tp
    
        fp = fp // cfg.w
        tn = tn // cfg.w
    return tp, tn, fp, fn, sigmas, (tp_ids, fp_ids, fn_ids, tn_ids)

In [17]:
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from show_loss import show_loss
from get_test_stats import get_test_stats
from show_test_data import show_test_data
from sklearn import metrics
from parameters import Stats
from IPython.display import clear_output
from config_reader import Config

'''
Train the network on given data.

nn           - Network to train.
batch_size   - Batch size to apply to data.
num_epochs   - How many epochs to train for.
train_data   - Data to train on.
labels_train - Labels for train_data.
loss_func    - Loss function.
optimizer    - Optimizer for gradients.
draw         - Whether to display loss dynamics.
step_test    - Whether to test on test data after each epoch.
args         - test data and test labels.

It's not recommended to use 'draw' and 'step_test' at the same time.
'''
def train(cnn, batch_size, num_epochs, train_data, labels_train_pre, loss_func, optimizer, args, draw=False, step_test=False, preload=False):
    if preload:
        try:
            cnn = models.resnet50(weights=models.ResNet50_Weights) 
            cnn.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            cnn.fc = nn.Linear(in_features=2048, out_features=2, bias=True)
            cnn.load_state_dict(torch.load("resnet50_full.pt"))
        except:
            cnn = models.resnet50(weights=models.ResNet50_Weights) 
            cnn.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            cnn.fc = nn.Linear(in_features=2048, out_features=2, bias=True) 
    cnn.train()
    cnn = cnn.double()
    loss_vals = []
    epoch_loss = []

    labels_train = torch.tensor([(0 if i == 2 else i) for i in labels_train_pre])
    
    test_stats_list = []
    train_stats_list = []

    tprs_test = []
    fprs_test = []
    tprs_train  = []
    fprs_train = []

    fprs_test.append(0)
    tprs_test.append(0)
    fprs_train.append(0)
    tprs_train.append(0)

    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        iters = int(train_data.shape[2] / batch_size)
        for i in range(iters):
            images = train_data[:, :, i:i + batch_size]
            labels = labels_train[i:i + batch_size]        
            
            b_x = images  
            # Change to Double if doesn't work
            b_y = labels.double()
            b_x = b_x.reshape((batch_size, 1, 36, 69))          
                
            optimizer.zero_grad()
            output = cnn(b_x)
            #print(output)
            loss = loss_func(input=output, target=b_y)   
            epoch_loss.append(loss.item())
            running_loss += loss.item() * batch_size
            
            if draw:
                show_loss(epoch_loss, loss_vals)
            elif (draw and i != 0 and i % 100 == 0 and not step_test and epoch > 0):
                show_loss(epoch_loss, loss_vals, clear_after=False)
                show_test_data(test_stats_list, train_stats_list)
                clear_output(True)
            elif (draw and i != 0 and i % 100 == 0 and not step_test and epoch == 0):
                show_loss(epoch_loss, loss_vals)
            
            loss.backward()                           
            optimizer.step()  

        if (step_test):
            tp, tn, fp, fn, sigmas, _ = test(cnn, args[0], args[1])  
            test_stats = get_test_stats(args[0].shape[2], tp, tn, fp, fn, sigmas)          
            test_stats_list.append(test_stats)          

            tp, tn, fp, fn, sigmas, _ = test(cnn, train_data, labels_train_pre)  
            train_stats = get_test_stats(args[0].shape[2], tp, tn, fp, fn, sigmas)       
            train_stats_list.append(train_stats)

            show_test_data(test_stats_list, train_stats_list)
            cnn.train()
        pass
        torch.save(cnn, "resnet50_full.pt")
        loss_vals.append(running_loss / train_data.shape[2])
        
    tprs_test += [item.tpr for item in test_stats_list]
    tprs_train += [item.tpr for item in train_stats_list]
    fprs_test += [item.fpr for item in test_stats_list]
    fprs_train += [item.fpr for item in train_stats_list]

    fprs_test.append(1)
    tprs_test.append(1)
    fprs_train.append(1)
    tprs_train.append(1)

    fprs_test, tprs_test = zip(*sorted(zip(fprs_test, tprs_test)))
    fprs_train, tprs_train = zip(*sorted(zip(fprs_train, tprs_train)))

    # sort by fpr
    auc_test = metrics.auc(fprs_test, tprs_test)
    auc_train = metrics.auc(fprs_train, tprs_train)
    return auc_test, auc_train 


In [20]:
models.resnet50()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
num_epochs = 40
batch_size = 24


y = torch.tensor(list(labels_train_old))
class_weights=class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
class_weights=torch.tensor(class_weights, dtype=torch.double)

cnn = models.resnet50(weights=models.ResNet50_Weights)
cnn.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
cnn.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1]/class_weights[0], reduction='mean') 
optimizer = optim.Adam(cnn.parameters(), lr=1e-6)

c:\Users\mike_live\.conda\envs\ml_climate\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\mike_live/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [19]:
auc_test, auc_train = train(cnn=cnn, batch_size=batch_size, num_epochs=num_epochs, train_data=train_data, labels_train_pre=labels_train, loss_func=criterion, optimizer=optimizer, args=(test_data, labels_test), draw=True, step_test=False, preload=False)

  0%|          | 0/40 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (24x2048 and 512x2)